----------2021-01-30----------------

author: shi_hailong

product: hai lingke

# 0、引入库

In [2]:
import serial
import threading
import time
from IPython.display import clear_output
import ctypes
import inspect

# 1、线程的结束代码

In [3]:
def _async_raise(tid, exctype):
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        raise SystemError("PyThreadState_SetAsyncExc failed")
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

# 2、配置串行通信

执行下面的语句之前一定要先在终端更改/dev/ttyTHS1的权限

1、sudo chmod 777 /dev/ttyTHS1

2、输入密码

In [6]:
xiaoke = serial.Serial("/dev/ttyTHS1",115200,timeout=1)
xiaoke.xonxoff = True
xiaoke.bytesize = serial.EIGHTBITS
xiaoke.stopbits = 1

# 000、临时的存储，因为不是专用的语音识别模块，所以先用一下，到后面定制了，就不需要这几行代码了

In [11]:
should_pronounce = {'openkongtiao':'forward','closekongtiao':'backward','guanbikaiguan':'stop', 'dakaizaoming':'left','guanbizhaoming':'right','guabdianfangbao':'middle'}

In [12]:
#a =['openkongtiao','closekongtiao','guanbikaiguan','dakaizaoming','guanbizhaoming','guabdianfangbao']

In [19]:
should_pronounce.values()
b = should_pronounce.keys()
print(b)
for i in b:
    print(i)

dict_keys(['openkongtiao', 'closekongtiao', 'guanbikaiguan', 'dakaizaoming', 'guanbizhaoming', 'guabdianfangbao'])
openkongtiao
closekongtiao
guanbikaiguan
dakaizaoming
guanbizhaoming
guabdianfangbao


# 3、解析语音识别的反馈信息

In [7]:
def getInfo():
    while True:
        while xiaoke.inWaiting()>0:
            try:
                myout=xiaoke.readline()
                clear_output(True)
                print(myout)
                s = str(myout)
                if 'open' in s:
                    print('----open----')
            except Exception as e:
                print('exception----')
                return 1

# 4、开始线程获取语音识别结果

In [8]:

t = threading.Thread(target=getInfo)
t.setDaemon(True)
t.start()

b'uArTcP\x02\x00\x03\xe9G\x1c\x00\r\xd1\xadclosekongtiao'


# 5、结束线程

In [9]:
stop_thread(t)